In [ ]:
%pip install torch torchvision clip-by-openai Pillow
%pip install sentence-transformers

In [8]:
import torch
import clip
from PIL import Image
from tkinter import Tk, filedialog
from tkinter.filedialog import askopenfilename
from sentence_transformers import SentenceTransformer, util

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
def preprocess_image(image_path):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    return image

In [5]:
def classify_image(image, descriptions):
    # Tokenize the descriptions
    text_inputs = clip.tokenize(descriptions).to(device)
    
    with torch.no_grad():
        # Encode the image and descriptions
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_inputs)

        # Calculate the similarity
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        
    probs = similarity.cpu().numpy()[0]
    best_idx = probs.argmax()
    return descriptions[best_idx], probs[best_idx]

In [6]:
descriptions = [
    "a photo of a beach",
    "a photo of a mountain",
    "a photo of a city",
    "a photo of a forest",
    "a photo of a desert",
    "a photo of a village"
]

In [11]:
image_path = filedialog.askopenfilename(title="Select an image", filetypes=[("Image files", "*.png *.jpg *.jpeg")])
if image_path:
    image = preprocess_image(image_path)
    predicted_label, confidence = classify_image(image, descriptions)
    print(f"Predicted location: {predicted_label} with confidence {confidence:.2f}")
else:
    print("No image selected.")

Predicted location: a photo of a village with confidence 0.26
